In [2]:
import pandas as pd
import os
from tqdm import tqdm

In [3]:
files = os.listdir("../tour_grade_time_series")
files = [os.path.join("../tour_grade_time_series", f) for f in files if os.path.isfile(os.path.join("../tour_grade_time_series", f))]


In [4]:
df = pd.DataFrame()

for file in tqdm(files):

    tmp = pd.read_csv(file)
    df = pd.concat([df, tmp], axis=0)

    print(df.shape)

  4%|▍         | 1/24 [00:01<00:25,  1.10s/it]

(3458438, 7)


  8%|▊         | 2/24 [00:02<00:33,  1.53s/it]

(8483159, 7)


 12%|█▎        | 3/24 [00:04<00:32,  1.56s/it]

...

(93841605, 7)


 96%|█████████▌| 23/24 [01:25<00:05,  5.21s/it]

(99229048, 7)


100%|██████████| 24/24 [01:31<00:00,  3.82s/it]

(103607977, 7)


In [5]:
df = df.fillna("")

In [6]:
df

,ProductCode,TourGradeCode,SeasonFrom,SeasonTo,week_day,start_time,adult_retail_price
0,102726P19,TG1,2020-03-11,2099-12-31,U,22:30,100.0
1,102726P19,TG2,2020-03-11,2099-12-31,F,21:30,135.0
2,102726P19,TG2,2020-03-11,2099-12-31,M,02:30,115.0
3,102726P19,TG1,2020-03-11,2099-12-31,M,04:30,833.34
4,102726P19,TG2,2020-03-11,2099-12-31,H,23:00,97.23
...,...,...,...,...,...,...,...
4378924,5049P50,TG8,2024-03-03,2025-02-28,F,10:00,39.0
4378925,5049P50,TG8,2024-03-03,2025-02-28,S,10:00,39.0
4378926,5049P50,TG8,2024-03-03,2025-02-28,T,10:00,39.0
4378927,5049P50,TG8,2024-03-03,2025-02-28,H,10:00,39.0


In [7]:
df['SeasonFrom'] = pd.to_datetime(df['SeasonFrom'])
df['SeasonTo'] = pd.to_datetime(df['SeasonTo'])

In [8]:
# Products should be active at least in 2022

cutoff_date = pd.to_datetime('2022-01-01')
df = df[df['SeasonTo'] >= cutoff_date]

# If future dates, keep current date as maximum

cutoff_date = pd.to_datetime('2024-07-01')
df.loc[df['SeasonTo'] > cutoff_date, 'SeasonTo'] = cutoff_date

# If start dates is in the future, keep current date as minimum

cutoff_date = pd.to_datetime('2024-07-01')
df.loc[df['SeasonFrom'] > cutoff_date, 'SeasonFrom'] = cutoff_date

In [9]:
df = df.drop_duplicates()

In [13]:
from pandas_gbq import to_gbq

In [1]:
df

NameError: name 'df' is not defined

In [14]:
df["adult_retail_price"] = df["adult_retail_price"].astype(str)

In [16]:
to_gbq(df, "v_extract1.product_price_time_series", project_id="ww-da-ingestion", if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 19328.59it/s]


In [2]:
import gc

import dash
import pandas as pd
import plotly.express as px
import yaml
from dash import dcc, html
from dash.dependencies import Input, Output

from mongodb_lib import *

config_infra = yaml.load(open("../infra-config-pipeline.yaml"), Loader=yaml.FullLoader)
db, fs, client = connect_to_mongodb(config_infra)

INFO:root:Successfully connected to MongoDB.


In [15]:
df = read_object(fs, "product_textual")
df = pd.DataFrame(df)

In [3]:
df_cont = read_object(fs, "product_textual_lang")
df_cont = pd.DataFrame(df_cont)

In [6]:
df_cont["pdt_product_detail_TOURGRADEDESCRIPTION"][0]

/tmp/ipykernel_1750473/2023272634.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_cont["pdt_product_detail_TOURGRADEDESCRIPTION"][0]


"['You can choose between two dates.- Sun, 9 Jun 2019 to Fri, 14 Jun 2019- Sun, 4 Aug 2019 to Fri, 9 Aug 2019\\n']"